# 基于GRU的字符生成

## GRU

门控循环单元（gated recurrent unit，GRU）是一种常用的门控循环神经网络

GRU引入了重置门（reset gate）和更新门（update gate）的概念，从而修改了循环神经网络中隐藏状态的计算方式。

![GRU](../Docs/GRU-Core.png)
<center>http://dprogrammer.org/rnn-lstm-gru</center>

* 重置门和更新门计算
$$ \begin{aligned} \boldsymbol{R}_t = \sigma(\boldsymbol{X}_t \boldsymbol{W}_{xr} + \boldsymbol{H}_{t-1} \boldsymbol{W}_{hr} + \boldsymbol{b}_r),\\
\boldsymbol{Z}_t = \sigma(\boldsymbol{X}_t \boldsymbol{W}_{xz} + \boldsymbol{H}_{t-1} \boldsymbol{W}_{hz} + \boldsymbol{b}_z), \end{aligned} $$
* 候选隐藏状态
$$\tilde{\boldsymbol{H}}_t = \text{tanh}(\boldsymbol{X}_t \boldsymbol{W}_{xh} + \left(\boldsymbol{R}_t \odot \boldsymbol{H}_{t-1}\right) \boldsymbol{W}_{hh} + \boldsymbol{b}_h),$$
* 隐藏状态更新
$$\boldsymbol{H}_t = \boldsymbol{Z}_t \odot \boldsymbol{H}_{t-1} + (1 - \boldsymbol{Z}_t) \odot \tilde{\boldsymbol{H}}_t.$$


**重置门有助于捕捉时间序列里短期的依赖关系.**

**更新门有助于捕捉时间序列里长期的依赖关系**

In [1]:
import time
import math
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F

import numpy as np
import pickle
import zipfile
import copy
import random

import utils as d2l

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## dataset

In [2]:
class TextTokenizer(object):
    def __init__(self, text=None, max_vocab=None, filename=None):
        if filename is not None:
            with open(filename, 'rb') as f:
                self.vocab = pickle.load(f)
        else:
            corpus_chars = set(text)
            # max_vocab_process
            vocab_count = {}
            for word in corpus_chars:
                if vocab_count.get(word) is None:
                    vocab_count[word] = 1
                else:
                    vocab_count[word] += 1

            vocab_count_list = []
            for word in vocab_count:
                vocab_count_list.append((word, vocab_count[word]))
            # sort count with number
            vocab_count_list.sort(key=lambda x: x[1], reverse=True)
            if max_vocab is not None and len(vocab_count_list) > max_vocab:
                vocab_count_list = vocab_count_list[:max_vocab]
            vocab = [x[0] for x in vocab_count_list]
            self.vocab = vocab

        self.char_to_index = {c: i for i, c in enumerate(self.vocab)}
        self.index_to_char = dict(enumerate(self.vocab))

    @property
    def vocab_size(self):
        return len(self.vocab) + 1

    def word_to_index(self, word):
        if word in self.char_to_index:
            return self.char_to_index[word]
        else:
            return len(self.vocab)

    def index_to_word(self, index):
        if index == len(self.vocab):
            return '<unk>'
        elif index < len(self.vocab):
            return self.char_to_index[index]
        else:
            raise Exception('Unknown index!')

    def text_to_array(self, text):
        arr = []
        for word in text:
            arr.append(self.word_to_index(word))
        return np.array(arr)

    def array_to_text(self, arr):
        words = []
        for index in arr:
            words.append(self.index_to_word(index))
        return "".join(words)

    def save_to_file(self, filename):
        with open(filename, 'wb') as f:
            pickle.dump(self.vocab, f)

In [3]:
# random sample
def data_loader_random(corpus_indices, seq_length, time_steps, device=None):
    num_samples = (len(corpus_indices) - 1) // time_steps
    epoch_size = num_samples // seq_length

    # shuffle sample
    samples_indices = list(range(num_samples))
    random.shuffle(samples_indices)

    # generator data
    for i in range(epoch_size):
        i = i * seq_length

        batch_incices = samples_indices[i: i + seq_length]
        x = [corpus_indices[indices: indices + time_steps] for indices in batch_incices]
        y = [corpus_indices[indices + 1: indices + time_steps + 1] for indices in batch_incices]

        x = torch.tensor(x, dtype=torch.float32).view(seq_length, time_steps)
        y = torch.tensor(y, dtype=torch.float32).view(seq_length, time_steps)

        yield x, y

In [4]:
# consecutive sample
def data_loader_consecutive(corpus_indices, seq_length, time_steps):

    data_len = len(corpus_indices)

    seq_size = data_len // seq_length

    # resize to => (seq_length, seq_size)
    corpus_indices = np.array(corpus_indices[: seq_size * seq_length], dtype=np.float).reshape((seq_length, -1))

    epoch_size = (seq_size - 1) // time_steps

    # generator data
    np.random.shuffle(corpus_indices)

    # convert to torch tensor
    torch_indices = torch.tensor(corpus_indices, dtype=torch.float32).view(seq_length, seq_size)
    for i in range(epoch_size):
        i = i * time_steps
        x = torch_indices[:, i: i + time_steps]
        y = torch_indices[:, i + 1: i + time_steps + 1]

        yield x, y

In [5]:
with zipfile.ZipFile('../Datasets/jaychou_lyrics/jaychou_lyrics.txt.zip') as zin:
    with zin.open('jaychou_lyrics.txt') as f:
        corpus_chars = f.read().decode('utf-8')  # corpus

corpus_chars = corpus_chars.replace('\n', ' ').replace('\t', ' ')
corpus_chars = corpus_chars[: 20000]

tokenizer = TextTokenizer(text=corpus_chars, max_vocab=None)

vocab = tokenizer.vocab
char_to_index = tokenizer.char_to_index
index_to_char = tokenizer.index_to_char
vocab_size = tokenizer.vocab_size

corpus_indices = tokenizer.text_to_array(corpus_chars)

## model

In [6]:
def onehot(x, n_class):
    # X shape: (batch, seq_len), output: seq_len elements of (batch, n_class)

    return [F.one_hot(x[:, i].to(torch.int64), n_class).to(dtype=torch.float32, device=device) for i in range(x.shape[1])]

X = torch.arange(10).view(2, 5)
inputs = onehot(X, vocab_size)
print(len(inputs), inputs[0].shape)

5 torch.Size([2, 1448])


In [7]:
def init_gru_state(seq_length, num_hiddens, device):
    return (torch.zeros((seq_length, num_hiddens), device=device),)

In [8]:
input_size, hidden_size, output_size = vocab_size, 256, vocab_size
# initial model params
def get_params():
    
    def _one(shape):
        ts = torch.tensor(np.random.normal(0, 0.01, size=shape), device=device, dtype=torch.float32)
        return torch.nn.Parameter(ts, requires_grad=True)

    def _three():
        return (_one((input_size, hidden_size)),
                _one((hidden_size, hidden_size)),
                nn.Parameter(torch.zeros(hidden_size, device=device, dtype=torch.float32),requires_grad=True))
    
    # hidden params
    w_xz, w_hz, b_z = _three() # update gate
    w_xr, w_hr, b_r = _three() # reset gate

    w_xh, w_hh, b_h = _three()  # candidate hidden state 
    
    # output params
    w_hq = _one((hidden_size, output_size))
    b_q = nn.Parameter(torch.zeros(output_size, device=device, dtype=torch.float32), requires_grad=True)
    
    return nn.ParameterList([w_xz, w_hz, b_z, w_xr, w_hr, b_r,  w_xh, w_hh, b_h, w_hq, b_q])

num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
print('will use', device)

will use cuda


In [9]:
def gru(inputs, states, params):

    """

    :param inputs: (time_steps, seq_len, vocab_size)
    :param states:
    :param params:
    :return:
    """

    (h,) = states
    w_xz, w_hz, b_z, w_xr, w_hr, b_r,  w_xh, w_hh, b_h, w_hq, b_q = params

    outputs = []

    for x in inputs:
        z_t = torch.sigmoid(torch.matmul(x, w_xz) + torch.matmul(h, w_hz) + b_z)
        r_t = torch.sigmoid(torch.matmul(x, w_xr) + torch.matmul(h, w_hr) + b_r)

        h_tilde = torch.tanh(torch.matmul(x, w_xh) + torch.matmul(r_t * h, w_hh) + b_h)
        
        h = z_t * h + (1 - z_t) * h_tilde
        
        y = torch.matmul(h, w_hq) + b_q
      
        outputs.append(y)
        
    return outputs, (h,)

In [10]:
X = torch.arange(10).view(2, 5)
state = init_gru_state(X.shape[0], hidden_size, device)
inputs = onehot(X.to(device), vocab_size)
print(inputs[0].size())
params = get_params()
outputs, state_new = gru(inputs, state, params)
print(len(outputs), outputs[0].shape, state_new[0].shape)

torch.Size([2, 1448])
5 torch.Size([2, 1448]) torch.Size([2, 256])


In [11]:
def grad_clipping(params, theta, device):
    norm = torch.tensor([0.0], device=device)
    for param in params:
        norm += (param.grad.data ** 2).sum()
    norm = norm.sqrt().item()
    if norm > theta:
        for param in params:
            param.grad.data *= (theta / norm)

In [12]:
def predict_gru(prefix, num_chars, gru, params, init_gru_state, 
                 num_hiddens, vocab_size, device, idx_to_char, char_to_idx):
    state = init_gru_state(1, num_hiddens, device)
    output = [char_to_idx[prefix[0]]]
    for t in range(num_chars + len(prefix) - 1):
        
        # previous last outputs as current input
        x = onehot(torch.tensor([[output[-1]]], device=device), vocab_size)
        # comuper outputs and state
        (y, state) = gru(x, state, params)
        
        # convert output from index to char
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(int(y[0].argmax(dim=1).item()))
    return ''.join([idx_to_char[i] for i in output])

In [13]:
p = predict_gru('想你', 10, gru, params, init_gru_state, hidden_size, vocab_size,
                    device, index_to_char, char_to_index)

print(p)

想你汗汗汗汗汗汗汗叫单登


In [14]:
def train_and_predict_gru(gru, get_params, init_gru_state, num_hiddens,
                          vocab_size, device, corpus_indices, idx_to_char,
                          char_to_idx, is_random_iter, num_epochs, num_steps,
                          lr, clipping_theta, seq_length, pred_period,
                          pred_len, prefixes):
    if is_random_iter:
        data_iter_fn = data_loader_random
    else:
        data_iter_fn = data_loader_consecutive
    params = get_params()
    loss = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        if not is_random_iter:  # 
            state = init_gru_state(seq_length, num_hiddens, device)
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = data_iter_fn(corpus_indices, seq_length, num_steps)
        for X, Y in data_iter:
            X = X.to(device)
            Y = Y.to(device)
            if is_random_iter:  
                state = init_gru_state(seq_length, num_hiddens, device)
            else:
                #
                for s in state:
                    s.detach_()

            inputs = onehot(X, vocab_size)
        
            (outputs, state) = gru(inputs, state, params)
        
            outputs = torch.cat(outputs, dim=0)
        
            y = torch.transpose(Y, 0, 1).contiguous().view(-1)
            # cross entropy
            l = loss(outputs, y.long())

            # gradient clearing
            if params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            l.backward()
            # gradient clearing
            grad_clipping(params, clipping_theta, device) 
            
            d2l.sgd(params, lr, 1)
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]

        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, math.exp(l_sum / n), time.time() - start))
            for prefix in prefixes:
                print(' -', predict_gru(prefix, pred_len, gru, params, init_gru_state,
                                        num_hiddens, vocab_size, device, idx_to_char, char_to_idx))

In [15]:
num_epochs, time_steps, seq_length, lr, clipping_theta = 250, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['爱你', '不爱']

In [16]:
# random sample
random_sample = True
train_and_predict_gru(gru, get_params, init_gru_state, hidden_size,
                      vocab_size, device, corpus_indices, index_to_char,
                      char_to_index, random_sample, num_epochs, time_steps, lr,
                      clipping_theta, seq_length, pred_period, pred_len,
                      prefixes)

epoch 50, perplexity 1.066066, time 0.52 sec
 - 爱你 漂亮的让我面红的可爱女人 温柔的让我心疼的可爱女人 透明的让我感动的可爱女人 坏坏的让我疯狂的可爱
 - 不爱女人 坏坏的让我疯狂的可爱女人 漂亮的让我面红的可爱女人 温柔的让我心疼的可爱女人 透明的让我感动的
epoch 100, perplexity 1.056030, time 0.52 sec
 - 爱你 漂亮的让我面红的可爱女人 温柔的让我心疼的可爱女人 透明的让我感动的可爱女人 坏坏的让我疯狂的可爱
 - 不爱女人 漂亮的让我面红的可爱女人 温柔的让我心疼的可爱女人 透明的让我感动的可爱女人 坏坏的让我疯狂的
epoch 150, perplexity 1.055549, time 0.52 sec
 - 爱你 温柔的让我心疼的可爱女人 透明的让我感动的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱
 - 不爱女人 温柔的让我心疼的可爱女人 透明的让我感动的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的
epoch 200, perplexity 1.054593, time 0.52 sec
 - 爱你 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 如果说怀疑 可以造句如果说分离 能够翻译 如果
 - 不爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 如果说怀疑 可以造句如果说分离 能够翻译 
epoch 250, perplexity 1.053257, time 0.53 sec
 - 爱你 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 漂亮的让我面红的可爱女人 温柔的让我心疼的可爱
 - 不爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 漂亮的让我面红的可爱女人 温柔的让我心疼的


In [17]:
# consecutive sample
random_sample = False
train_and_predict_gru(gru, get_params, init_gru_state, hidden_size,
                      vocab_size, device, corpus_indices, index_to_char,
                      char_to_index, random_sample, num_epochs, time_steps, lr,
                      clipping_theta, seq_length, pred_period, pred_len,
                      prefixes)

epoch 50, perplexity 58.862016, time 0.55 sec
 - 爱你 我不能再想 我不要再想 我不要我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我
 - 不爱 我不能再想 我不要再想 我不要我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我
epoch 100, perplexity 4.292827, time 0.53 sec
 - 爱你走过我好多 好多下耳 就知着我 你知着我不好重 我不能再想 我不要再想 我不 我不 我不要再想你 不
 - 不爱 你在在角中 有一个人气 家物苦斤的牛肉 让我们 半兽人 的灵魂 单纯 对远古存在的兽 用谦卑的半份
epoch 150, perplexity 1.407988, time 0.53 sec
 - 爱你怎么我要要的一定 从不再说 泪离的泪离 是何检的旧叫 闪通名风豆 爷经的父丽过 帅何了我 全场盯人防
 - 不爱 在水中中眼离猜 我害怕的想有有样元不了了活 一直是那再已一点糜 他看到过去 试着让你怎么快找 不安
epoch 200, perplexity 1.110745, time 0.55 sec
 - 爱你比过我较多的爸口 河边的风 在小着头发飘动 牵着你的手 一阵莫名感动 我想带你 回我的外婆家 一起看
 - 不爱 藤水止中带过我 抛入心危梦 登上泰梦的你 得思寄的豆叫画口 无地千年的转生 维地安老斑象的民谣 和
epoch 250, perplexity 1.069525, time 0.52 sec
 - 爱你比何我 说不休 语沉默怎怎么是子汉手 心什么这么简单简详 不懂得酒 如果这一切 真的可以 我想要将我
 - 不爱 藤非止中在最不到稳 我的等后将被摧毁 他许颓与愿违 累论累　风就平日难平  我们到伊斯坦堡 我才会
